# Проект: "Рекомендация тарифов"

# Описание проекта 

 
        
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:

        сalls — количество звонков,
        minutes — суммарная длительность звонков в минутах,
        messages — количество sms-сообщений,
        mb_used — израсходованный интернет-трафик в Мб,
        is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
        
Нужно построить модель, которая выберет подходящий тариф.         


In [8]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

# Открыть и изучить данные.

In [9]:
df = pd.read_csv('/datasets/users_behavior.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


        Целевой признак является категориальным, значит предстоит решить задачу классификации.
        Видим, что нет пропусков, все форматы соответствуют, также видим два столбика 'calls' и 'minutes', 
        котрые отражают объем использвания разговоров в тарифах.
        Подтвердим или опровергнем наличие их зависимоти с помощью измерения корреляции.

In [10]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Связь двух параметров подтвердилась. Коэф.корреляции calls/minutes =  0.98.


# Разделим исходные данные на обучающую, валидационную и тестовую выборки.

Разделим датасет на выборки в соотношении 60%/ 20%/ 20%  .

In [11]:
df_train, df_valid, df_test = np.split(df, [int(.6*len(df)), int(.8*len(df))])

     Сохраненим признаки в отдельные переменные features и target.
     Не будем испоьзовать столбец 'calls', тк включение обоих параметров('calls' и 'minutes') во множество
     признаков для моделирования приведет к мультиколлинеарности. Оставляем 'minutes', тк обычно в пакет тарифов
     включают количество минут, а не звонков.

In [12]:
features_train = df_train.drop(['is_ultra', 'calls'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra', 'calls'], axis=1)
target_valid  = df_valid['is_ultra']

features_test= df_test.drop(['is_ultra', 'calls'], axis=1)
target_test = df_test['is_ultra']

# Исследуем качество разных моделей

Построим разные модели.

In [16]:
#Дерево решений
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth) 
    model.fit(features_train, target_train)
    accuracy_train = model.score(features_train, target_train)
    accuracy_valid = model.score(features_valid, target_valid)
        
    if accuracy_valid > best_result:
        best_result = accuracy_valid
        best_depth = depth
        
print("Accuracy лучшей модели на валидационной выборке:", best_result, 'Глубина дерева:', best_depth)

Accuracy лучшей модели на валидационной выборке: 0.8118195956454122 Глубина дерева: 9


Модель "Дерево решений" показала наилучшие результаты при глубине дерева = 9.


In [17]:
# Random forest
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 60, 5):
    for depth in range (1, 11):
        model_rand = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth= depth)
        model_rand.fit(features_train, target_train)
        accuracy_train = model_rand.score(features_train, target_train)
        accuracy_valid = model_rand.score(features_valid, target_valid)
        if accuracy_valid > best_result:
            best_result = accuracy_valid
            best_est = est
            best_depth = depth       

print("Accuracy лучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)

Accuracy лучшей модели на валидационной выборке: 0.8149300155520995 Количество деревьев: 45 Максимальная глубина: 10


Модель "Случайный лес" показала наилучшие результаты при количетве деревьев = 45 и глубине = 10.

In [15]:
#Логистическая регрессия

parametrs = ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']
best_result = 0
for param in parametrs:
    model_log = LogisticRegression(random_state=123456, solver=param, max_iter=200) 
    model_log.fit(features_train, target_train)
    accuracy_valid = model_log.score(features_valid, target_valid)
   #accuracy_test = model_log.score(features_test, target_test)
    if accuracy_valid > best_result:
        best_result = accuracy_valid
        best_solver = param
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Алгоритм:", best_solver)        

Accuracy наилучшей модели на валидационной выборке: 0.7480559875583204 Алгоритм: newton-cg


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

Модель "Логистическая регрессия" показала наилучшие результаты при использовании алгоритма "newton-cg".

    ВОПРОС:
    
        'newton-cholesky'
        ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got newton-cholesky.
        Почему так? Что-то надо подкрутить в версиях? Каких

Вывод: Наилучшей моделью оказалась "Случайный лес", accuracy на валидационной выборке = 0.8149300155520995.
    

# Проверим качество модели на тестовой выборке

In [382]:
model_rand = RandomForestClassifier(random_state=12345, n_estimators=45, max_depth= 10)
model_rand.fit(features_train, target_train)
accuracy_test_model_rand = model_rand.score(features_test, target_test)
accuracy_test_model_rand

0.8180404354587869

Мы видим, что на тестовой выборке модель предсказывает не хуже, чем на валидационной. Это хороший результат, оставляем эту модель.

# Дополнительное задание: проверьте модели на вменяемость.

Выполним проврку, используя сравнение с часто встречающимся классом. В данных чаще всего встречается тариф Smart. 
Оценим его долю.

In [364]:
df.query('is_ultra == 0')
#2229 строк из 3214
part_0 = 2229/3214
part_0

0.693528313627878

Получили 69% - 0, тариф Smart
Если модель неадекватна и будет выдавать одни нули, она угадает 69% тарифов,
тогда accuracy такой кривой модели = 0.69. 
Мои модели имеют accuracy выше.
Таким образом все три модели по этому критерию соответсвуют вменяемым моделям.